# SAP Automation using Python

In [5]:
import win32com.client
import subprocess
import time

In [6]:
# # warning to display pop up message
# def warning():
#     from tkinter import messagebox
#     root = messagebox.showwarning("Warning", "This Automation bot is to view the material details which are already existing!")
#     # time for this message to stay
#     time.sleep(2)
#     # root.mainloop()
#     root.destroy()
# warning()

In [7]:
# open SAP Logon
subprocess.Popen(r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection("USDEV",True)

# login
session = connection.Children(0)
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = "800" # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "JESAI"    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "jaisai2025"  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [8]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nmm03" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [9]:
# Entering Material Number
session.findById("wnd[0]/usr/ctxtRMMG1-MATNR").text = "7100031" # Material Number
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)
